Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title 🛠️ Environment Setup (completo, IPAdapterPlus incluido y limpio)
OPTIONS = {}
USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

# Montar Google Drive (opcional)
if OPTIONS['USE_GOOGLE_DRIVE']:
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"

# Ir a /content
%cd /content

# Clonar ComfyUI si no existe
!if [ ! -d "$WORKSPACE" ]; then echo "Clonando ComfyUI..."; git clone https://github.com/comfyanonymous/ComfyUI.git $WORKSPACE; fi

# Entrar al workspace
%cd $WORKSPACE

# Actualizar si se desea
if OPTIONS['UPDATE_COMFY_UI']:
    !echo "-= Actualizando ComfyUI =-"
    !git pull

# Instalar carpeta custom_nodes y clonar IPAdapterPlus (limpio)
!mkdir -p custom_nodes
%cd custom_nodes
!rm -rf ComfyUI_IPAdapter_plus
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git
%cd ..

# Instalar dependencias necesarias
!echo "-= Instalando dependencias =-"
!pip install --upgrade pip
!pip install torch==2.7.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install xformers==0.0.31 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121
!pip install open-clip-torch einops ftfy opencv-python tqdm

print("✅ Setup completo. Continuá con la celda 2 para bajar los modelos.")

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

In [10]:
# Crear carpetas necesarias
!mkdir -p ./models/ipadapter/
!mkdir -p ./models/checkpoints/
!mkdir -p ./models/vae/
!mkdir -p ./models/clip_vision/
!mkdir -p ./models/controlnet/
!mkdir -p ./models/upscale_models/
!mkdir -p ./input/
!mkdir -p ./output/dataset/

# ================================
# MODELOS BASE (Los que ya tienes)
# ================================

# Checkpoint de Stable Diffusion 1.5
!wget -c https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors -P ./models/checkpoints/

# VAE recomendado
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

# IPAdapter básico (general)
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin -P ./models/ipadapter/

# IPAdapter rostro avanzado (RECOMENDADO para consistency facial)
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors -P ./models/ipadapter/

# CLIPVision correcto para IPAdapter SD1.5
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/pytorch_model.bin -O ./models/clip_vision/pytorch_model.bin

# ================================
# MODELOS ADICIONALES PARA DATASET
# ================================

# Checkpoint más realista (OPCIONAL - mejor para fotos)
!wget -c https://huggingface.co/digiplay/majicMIX_realistic_v7/resolve/main/majicmixRealistic_v7.safetensors -P ./models/checkpoints/

# ControlNet para consistencia de pose
!wget -c https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth -P ./models/controlnet/

# Upscaler para mejorar calidad
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/

# ================================
# NODOS ADICIONALES NECESARIOS
# ================================

# Instalar ControlNet nodes
!cd custom_nodes && git clone https://github.com/Fannovel16/comfyui_controlnet_aux.git
!cd custom_nodes && git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git

# Instalar nodos de utilidad
!cd custom_nodes && git clone https://github.com/rgthree/rgthree-comfy.git

# VERIFICAR ARCHIVOS
print("✅ Verificando archivos descargados...")
import os

required_files = {
    "Checkpoint SD1.5": "./models/checkpoints/v1-5-pruned-emaonly-fp16.safetensors",
    "VAE": "./models/vae/vae-ft-mse-840000-ema-pruned.safetensors",
    "IP-Adapter básico": "./models/ipadapter/ip-adapter_sd15.bin",
    "IP-Adapter Face": "./models/ipadapter/ip-adapter-plus-face_sd15.safetensors",
    "CLIP Vision": "./models/clip_vision/pytorch_model.bin"
}

for name, path in required_files.items():
    if os.path.exists(path):
        size = os.path.getsize(path) / (1024*1024)  # MB
        print(f"✅ {name}: {size:.1f} MB")
    else:
        print(f"❌ {name}: FALTANTE")

print("\n🎯 SETUP COMPLETO PARA DATASET GENERATION")
print("📂 Coloca tu imagen de referencia en: ./input/")
print("🖼️ Las imágenes generadas se guardarán en: ./output/dataset/")


--2025-06-26 19:29:19--  https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.80, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f5/2e/f52e0ca18fd9491c78516fb4c4b063c960ed7a3b061f667ff8730433c643cf05/e9476a13728cd75d8279f6ec8bad753a66a1957ca375a1464dc63b37db6e3916?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27v1-5-pruned-emaonly-fp16.safetensors%3B+filename%3D%22v1-5-pruned-emaonly-fp16.safetensors%22%3B&Expires=1750969759&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDk2OTc1OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Y1LzJlL2Y1MmUwY2ExOGZkOTQ5MWM3ODUxNmZiNGM0YjA2M2M5NjBlZDdhM2IwNjFmNjY3ZmY4NzMwNDMzYzY0M2NmMDUvZTk0NzZhMTM3MjhjZDc1ZDgyN

### Run ComfyUI with cloudflared (Recommended Way)




In [11]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-06-26 19:30:33--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.6.1/cloudflared-linux-amd64.deb [following]
--2025-06-26 19:30:33--  https://github.com/cloudflare/cloudflared/releases/download/2025.6.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/f2a50f07-6f2f-47cd-8a16-5d3dca82800f?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250626%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250626T193033Z&X-Amz-Expires=1800&X-Amz-Signature=e29f327a2b10b1d2dc5bb7e3db925a64d3662386d0386d2d44c61c1a07a

### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

import threading

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server